In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('CP - Resultados - Main.csv')

# Convert time to hours
def time_to_hours(time_str):
    try:
        h, m, s = map(int, time_str.strip().split(':'))
        return (h * 3600 + m * 60 + s) / 3600
    except:
        return np.nan

df['Time_hours'] = df['Time'].apply(time_to_hours)
df = df.dropna(subset=['Time_hours', 'Acc'])

# Filter datasets for CIFAR10 and CIFAR100
df_cifar10 = df[df['Dataset'] == 'CIFAR10']
df_cifar100 = df[df['Dataset'] == 'CIFAR100']

# Define architectures and their colors
architecture_colors = {
    'RESNET44': 'blue',
    'RESNET56': 'orange',
    'RESNET110': 'red'
}

# Function to plot data for a specific architecture
def plot_architecture(arch, color):
    for dataset_name, dataset in zip(['CIFAR10', 'CIFAR100'], [df_cifar10, df_cifar100]):
        arch_data = dataset[dataset['Architecture'] == arch]
        no_subsample = arch_data[arch_data['Subsampling'] == 'Sem']
        with_subsample = arch_data[arch_data['Subsampling'] == 'Com']
        
        # Create a new figure
        plt.figure(figsize=(7, 6))
        
        # Plot points
        plt.scatter(no_subsample['Time_hours'], no_subsample['Acc'], color=color, marker='s', label='No Subsampling')
        plt.scatter(with_subsample['Time_hours'], with_subsample['Acc'], color=color, marker='^', label='Subsampling')
        
        # Highlight points with missing CP or EP Ann.
        missing_points = arch_data[(arch_data['CP'] == '-') & (arch_data['EP Ann.'] == '-')]
        plt.scatter(missing_points['Time_hours'], missing_points['Acc'], facecolor='none', edgecolor='red', marker='o', s=100, linewidth=1.5, label='Baseline')
        
        # Add title, labels, and legend
        plt.title(f'{arch} - {dataset_name}')
        plt.xlabel('Time (hours)')
        plt.ylabel('Accuracy')
        plt.grid(True)
        plt.legend(fontsize='small')
        
        # Save the figure as a PDF
        plt.tight_layout()
        plt.savefig(f'./figures/{arch}_{dataset_name}_time_vs_performance.pdf')
        plt.show()

# Generate a separate graph for each architecture
for arch, color in architecture_colors.items():
    plot_architecture(arch, color)
